In [153]:
import requests
from bs4 import BeautifulSoup
import numpy as np

In [154]:
documents = []
pages = []
for idx in range(43):
    url = "https://www.overleaf.com/articles/tagged/math/page/{}".format(idx + 1)
    r = requests.get(url)
    if (r.status_code == 200):
        soup = BeautifulSoup(r.content, "html.parser")
        pages += [soup]
        articles = soup.find_all('div', class_='template-thumbnail')
        for article in articles:
            art = requests.get("https://www.overleaf.com" + article.a.get_attribute_list("href")[0])
            if (art.status_code == 200):
                soup_art = BeautifulSoup(art.content, "html.parser")
                documents += [soup_art.find('script', id="viewSourceModalTemplate")]
            else:
                print("Get error article:", article)
    else:
        print("Get error page:", idx)

In [172]:
print(documents[200].get_text().find("\documentclass"))

450


In [175]:
latex_documents = []
for doc in documents:
    if doc is None:
        continue
    begin = doc.get_text().find("\documentclass")
    end = doc.get_text().find("\end{document}") + 14
    latex_documents += [doc.get_text()[begin:end]]

In [185]:
import pandas as pd

df = pd.DataFrame(latex_documents, columns=['content'])
df.to_csv("LaTeX_articles.csv", index=False)